<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>

# OpenEnergyPlatform
<br><br>

# API tutorial 1 - Introduction to the OEP-API and basic table opperations
Repository: https://github.com/openego/oedialect

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/examples/issues <br>
How to get started with Jupyter Notebooks can be found here: https://github.com/OpenEnergyPlatform/oeplatform/wiki <br>
Please ensure you have read the Terms of use here: https://openenergy-platform.org/legal/tou/

In [1]:
__copyright__ = "Reiner Lemoine Institut, Zentrum für nachhaltige Energiesysteme Flensburg"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "wolfbunke, Ludee"

## Introduction

<br>
<div class="alert alert-block alert-danger">
This is an important information!
</div>
<div class="alert alert-block alert-info">
This is an information!
</div>
<div class="alert alert-block alert-success">
This is your task!
</div>

This tutorial gives you an overview of the [**OpenEnergy Platform**](https://openenergy-platform.org/) and how you can work with the **REST-ful-HTTP** API in Python. <br>


## Part I - How to work with the OpenEnergy Platform (OEP)

  Get started - Sign-in and get your own token <br>
0 Setup token <br>
1 Create a table <br>
2 Delete a table <br>
3 Query table columns <br>
4 Insert Data into a table <br>
5 Insert data into a specific row <br>
6 Alter data in a table <br>


## Links and further information

How to work with Python and Jupyter Notebooks <br>
[OPSD wiki](https://github.com/Open-Power-System-Data/common/wiki) <br>
[nbviewer example](http://nbviewer.jupyter.org/github/ocefpaf/folium_notebooks/blob/master/test_png_mpld3_vega_popup.ipynb)


#### Overview of Packages:
http://geopandas.org/install.html#installing-geopandas

# Part I
 
## 0. Get started - Sign-in and get your own token

Register a new account on the [**OpenEnergyPlatform**](https://openenergy-platform.org/user/register:RequestAccount). Then login (https://openenergy-platform.org/).


#### 1.  Click on the login button and sign in

<img src="images/login.png"  alt="OpenEnergy Platform login" height="100" width="100" align="left"/>  <br><br>


#### 2.  Copy your token

Click on your Profile Name to see your information. To view your token, click on "Show token" 

<img src="images/token.png"  alt="OpenEnergy Platform Token" height="400" width="500" align="left" />
<br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br>

#### 3a.  Paste your token in an environment variable

The variable should be named 'OEP_TOKEN'. 
On linux :
```
export OEP_TOKEN=<your token>
```
On windows:
```
set OEP_TOKEN=<your token>
```

#### 3b.  Paste your token in a file (discouraged)

Look for the following line in the file `api/token_config.py`,

```python
OEP_TOKEN = ''
```
then paste your token there

```python
OEP_TOKEN = '<paste_your_token_here>'
```

Save the file but DO NOT PUSH the change to GitHub

## 0. Setup token

<br>
<div class="alert alert-block alert-danger">
Do not push your token to GitHub!
</div>

In [ ]:
import requests
import pandas as pd
from IPython.core.display import HTML

from token_config import oep_url, get_oep_token

# token
your_token = get_oep_token()

## 0.1 About the Database and used Packages
Data are stored and ordered by topic under the "schema" tab:
https://openenergy-platform.org/dataedit/schemas  </div> 

The data tables can be accessed by calling them via http request. </div> 


The "request" package will handle the HTTP communication with the OEP database. It sends requests that insert (put, post, delete) or read (get) data in/from the databank. For either request the URL of a requested table has to be included. 
For all "put", "post" and "delete" requests one has to include the authentification token as a header, like in the "delete table " example.</div> 



Example:
```python
requests.delete(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table, 
    headers={'Authorization': 'Token %s'%your_token}
)
```

**About the table structure:**  <br>

The tables are organized in rows and columns that can be adressed via: <br>





```python
requests.post(oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/')
# and       
requests.get(oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/columns')
```

In the following tutorial the schema will be 'model_draft'and the table 'example_api_table_test'.
    
Inserted data must have the structure of a dictionary / json with a key for every column of the table and the respective value like in: 

Random example from [OpenStreetMap](https://www.openstreetmap.de/karte.html?zoom=15&lat=52.40731&lon=7.46118&layers=B000TT) with freely chosen capacity.

In [ ]:
data = {
    "query": {
        "name": "Windpark, Am Speller Sand", 
        "type":  "wind_onshore",
        "capacity": 20000,
        "lat": 52.40731,
        "lon": 7.46118
    }
}

## 1. Create a table / Table Architecure 



You installed all Python packages? Let's create our first database table:

<div class="alert alert-block alert-info">
The API is enabled for the following schmemas only: <b>'model_draft'</b> & <b>'sandbox'</b>
</div>
<div class="alert alert-block alert-info">
A table must have a column <b>'id'</b> of type <b>'bigserial'</b>!
</div>
<div class="alert alert-block alert-success">
Change the name of the table!
</div>

In [ ]:
# create table
schema = 'model_draft'
table = 'example_api_table_test'

data = { 
    "query": { 
        "columns": [
            { "name":"id", "data_type": "bigserial", "is_nullable": "NO" },
            { "name":"name", "data_type": "varchar", "character_maximum_length": "50" },
            { "name":"type", "data_type": "varchar", "character_maximum_length": "20" },
            { "name":"capacity", "data_type": "decimal" },
            { "name":"lat", "data_type": "numeric" },
            { "name":"lon", "data_type": "numeric" }
        ],
        "constraints": [ { "constraint_type": "PRIMARY KEY", "constraint_parameter": "id" } ] 
    }
}

requests.put(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/',
    json=data,
    headers={'Authorization': 'Token %s'%your_token}
)

<div class="alert alert-block alert-info">
<b>Response [201]</b> succesfully created table! <br>
<b>Response [400]</b> table already exists! (choose another name for table)
</div>

In [ ]:
# check if your table exists
requests.get(oep_url + '/api/v0/schema/' + schema + '/tables/' + table)

<div class="alert alert-block alert-info">
<b>Response [200]</b> table exists! <br>
<b>Response [404]</b> table doesn't exist!
</div>

Table can be seen online:
https://openenergy-platform.org/dataedit/view/model_draft/example_api_table_test or https://openenergy-platform.org/dataedit/view/model_draft/ + name_of_your_table

## 2. Delete a table

In [ ]:
# Delete your table
requests.delete(oep_url + '/api/v0/schema/' + schema + '/tables/' + table, headers={'Authorization': 'Token %s'%your_token} )

<div class="alert alert-block alert-info">
<b>Response [200]</b> succesfully deleted table! <br>
<b>Response [404]</b> table doesn't exist and cannot be deleted!
</div>

In [ ]:
# check if your table was properly deleted
requests.get(oep_url + '/api/v0/schema/' + schema + '/tables/' + table)

<div class="alert alert-block alert-info">
<b>Response [404]</b> table does not exists!
</div>

<div class="alert alert-block alert-success">
Now create the table again! This is important for the following tutorials! 
</div>

In [ ]:
# recreate the same table as above
requests.put(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/',
    json=data, 
    headers={'Authorization': 'Token %s'%your_token}
)

## 3. Query table columns

In [ ]:
result = requests.get(oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/columns')
json_result = result.json()
json_result['capacity']

# all:
#json_result

## 4. Insert Data into a table

### 4.1 Insert single row

Now we insert a power plant with name, type and latitude longitude cooridinates.

In [ ]:
data = {
    "query": {
        "name": "Anlage 1", 
        "type": "wind_onshore",
        "capacity": 20000,
        "lat": 52.40731,
        "lon": 7.46118
    }
}

requests.post(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/new',
    json=data,
    headers={'Authorization': 'Token %s'%your_token}
)

<div class="alert alert-block alert-info">
<b>Response [201]</b> succesfully inserted data! <br>
<b>Response [404]</b> table doesn't exist!
</div>

### 4.1 Insert multiple rows
If you want to insert multiple lines of data like two power plants in this case, you can insert a list of dictionaries with as many entries as the table has columns. 

In [ ]:
data = {
    "query": [
        {
            "name": "Anlage 2", 
            "type": "photovoltaics",
            "capacity": 10,
            "lat": 51.804783,
            "lon": 11.686346
        },
        {
            "name": "Anlage 3", 
             "type": "photovoltaic", 
             "capacity": 5.5,
             "lat": 51.804783,
             "lon": 11.6341573
        }
    ]
}

requests.post(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/new', 
    json=data,
    headers={'Authorization': 'Token %s'%your_token}
)

<div class="alert alert-block alert-info">
<b>Response [201]</b> succesfully inserted data! <br>
<b>Response [404]</b> table doesn't exist!
</div>

## 5. Insert data into a specific row

In [ ]:
# insert data to row 11
rownumber = '11'

data = {
    "query": {
        "name": "Anlage 11",
        "type": "photovoltaic",
        "capacity": 5.5,
        "lat": 51.804783,
        "lon": 10.41573
    }
}

requests.put(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/' + rownumber, 
    json=data,
    headers={'Authorization': 'Token %s'%your_token}
)

<div class="alert alert-block alert-info">
<b>Response [201]</b> succesfully inserted data! <br>
<b>Response [200]</b> row already taken! <br>
<b>Response [404]</b> table doesn't exist!
</div>

<div class="alert alert-block alert-success">
Insert some powerplants!
</div>

In [ ]:
# insert some data




## 6. Alter data in a table

In [ ]:
requests.get(oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/') # Load the names via GET

<div class="alert alert-block alert-info">
<b>Response [200]</b> table exists! <br>
<b>Response [404]</b> table doesn't exist!
</div>

In [ ]:
row = 2
data = {"query": {"capacity": "2", "type": "pv"}}

requests.post(
    oep_url + '/api/v0/schema/' + schema + '/tables/' + table + '/rows/{id}'.format(id=row),
    json=data,
    headers={'Authorization': 'Token %s'%your_token}
)

<div class="alert alert-block alert-info">
<b>Response [200]</b> succesfully changed data! <br>
<b>Response [404]</b> table doesn't exist!
</div>